In [6]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import json

In [2]:
def get_data(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    balance_sheet = stock.balance_sheet
    cashflow = stock.cashflow
    return financials, balance_sheet, cashflow

In [4]:
def calculate_piotroski_fscore(financials, balance_sheet, cashflow):
    # Ensure the data is ordered correctly
    financials = financials.T.sort_index()
    balance_sheet = balance_sheet.T.sort_index()
    cashflow = cashflow.T.sort_index()

    # Initialize score
    score = 0

    # Profitability
    if 'Net Income' in financials.columns and 'Total Assets' in balance_sheet.columns:
        roa = financials['Net Income'] / balance_sheet['Total Assets']
        score += (roa.iloc[-1] > 0)  # Positive ROA
        score += (roa.iloc[-1] > roa.iloc[-2])  # ROA improvement

    if 'Operating Cash Flow' in cashflow.columns:
        score += (cashflow['Operating Cash Flow'].iloc[-1] > 0)  # Positive Operating Cash Flow

    if 'Net Income' in financials.columns and 'Operating Cash Flow' in cashflow.columns and 'Total Assets' in balance_sheet.columns:
        accrual = cashflow['Operating Cash Flow'] / balance_sheet['Total Assets']
        score += (accrual.iloc[-1] > 0)  # Accrual

    # Leverage, Liquidity, and Source of Funds
    if 'Long Term Debt' in balance_sheet.columns:
        delta_leverage = balance_sheet['Long Term Debt'].iloc[-1] - balance_sheet['Long Term Debt'].iloc[-2]
        score += (delta_leverage <= 0)  # Decrease in Leverage

    if 'Current Assets' in balance_sheet.columns and 'Current Liabilities' in balance_sheet.columns:
        current_ratio = balance_sheet['Current Assets'] / balance_sheet['Current Liabilities']
        score += (current_ratio.iloc[-1] > current_ratio.iloc[-2])  # Improved Current Ratio

    if 'Common Stock' in balance_sheet.columns:
        delta_shares = balance_sheet['Common Stock'].iloc[-1] - balance_sheet['Common Stock'].iloc[-2]
        score += (delta_shares <= 0)  # No new equity issuance

    # Operating Efficiency
    if 'Total Revenue' in financials.columns and 'Cost Of Revenue' in financials.columns and 'Total Assets' in balance_sheet.columns:
        gross_margin = (financials['Total Revenue'] - financials['Cost Of Revenue']) / financials['Total Revenue']
        score += (gross_margin.iloc[-1] > gross_margin.iloc[-2])  # Improved Gross Margin

        asset_turnover = financials['Total Revenue'] / balance_sheet['Total Assets']
        score += (asset_turnover.iloc[-1] > asset_turnover.iloc[-2])  # Improved Asset Turnover

    return score

In [11]:
#Read companies dict from the json file
with open('companies.json', 'r') as f:
    companies = json.load(f)

In [12]:
# Loop through each company to calculate the Piotroski F-score
results = []

for company in companies:
    financials, balance_sheet, cashflow = get_data(company["ticker"])
    score = calculate_piotroski_fscore(financials, balance_sheet, cashflow)
    results.append({
        "Company": company["name"], 
        "Ticker": company["ticker"], 
        "Country": company["country"], 
        "Industry": company["industry"],
        "Piotroski F-score": score
    })


In [13]:
results_df = pd.DataFrame(results)
results_df

,Company,Ticker,Country,Industry,Piotroski F-score
0,Microsoft Corporation,MSFT,USA,Technology,5
1,Alibaba Group Holding Limited,BABA,China,E-commerce,8
2,Nestlé S.A.,NSRGY,Switzerland,Consumer Goods,7
3,Toyota Motor Corporation,TM,Japan,Automotive,6
4,Johnson & Johnson,JNJ,USA,Healthcare,8
...,...,...,...,...,...
86,"Nintendo Co., Ltd.",NTDOY,Japan,Technology,4
87,"Hitachi, Ltd.",HTHIY,Japan,Technology,4
88,Panasonic Corporation,PCRFY,Japan,Technology,0
89,Mitsubishi Electric Corporation,MIELY,Japan,Technology,6


In [14]:
sorted_df = results_df.sort_values(by='Piotroski F-score', ascending=False)
sorted_df

,Company,Ticker,Country,Industry,Piotroski F-score
75,"Square, Inc.",SQ,USA,Financials,9
73,"Salesforce.com, Inc.",CRM,USA,Technology,9
81,NVIDIA Corporation,NVDA,USA,Technology,9
62,Halliburton Company,HAL,USA,Energy,9
74,"PayPal Holdings, Inc.",PYPL,USA,Financials,8
...,...,...,...,...,...
27,Royal Dutch Shell plc,RDS-A,Netherlands,Energy,0
70,"Facebook, Inc.",FB,USA,Technology,0
85,LG Electronics Inc.,LGEAF,South Korea,Technology,0
88,Panasonic Corporation,PCRFY,Japan,Technology,0


In [16]:
#Save the result to a csv file
sorted_df.to_csv("Diversed-Companies-Sorted-by-F-score.csv")